# Finding and Ordering Imagery for the Largest Active Fire on Earth

In this notebook, we'll combine NASA's FIRMS (Fire Information for Resource Management System) active fire data with Google Earth Engine and Planet imagery to:

1. **Load today's active fires** from the MODIS or VIIRS instruments via FIRMS
2. **Identify the largest fire** by grouping nearby fire pixels into connected clusters
3. **Polygonize the fire boundary** to create an area of interest (AOI)


This workflow demonstrates real-world remote sensing analysis: detecting an event from coarse hotspot data, defining a precise AOI, and acquiring high-resolution commercial imagery for detailed assessment.

## APIs and Tools

- **[Google Earth Engine Python API](https://developers.google.com/earth-engine/guides/python_install)** – for geospatial analysis and data processing
- **[geemap](https://geemap.org/)** – interactive mapping library for Earth Engine in Python
- **[Planet SDK for Python](https://planet-sdk-for-python-v2.readthedocs.io/)** – for programmatic access to Planet's data catalog
- **[Planet Orders API](https://developers.planet.com/docs/apis/orders/)** – for ordering and delivering Planet imagery

## Datasets

- **[FIRMS Active Fire Data](https://firms.modaps.eosdis.nasa.gov/)** – near real-time fire detections from MODIS and VIIRS instruments
- **[PlanetScope Imagery](https://developers.planet.com/docs/data/planetscope/)** – high-resolution (3–5 m) daily imagery from Planet's satellite constellation

## Prerequisites

- Authenticated access to Google Earth Engine (`ee.Authenticate()` and `ee.Initialize()`)
- A Planet API key (sign up at [planet.com](https://www.planet.com/) for a free trial or educational account)
- Python packages: `earthengine-api`, `geemap`, `planet`, `requests`, `geopandas`, `shapely`

Let's get started!

## Installing Required Libraries

Before we begin, we need to install the necessary Python packages for this workflow:

- **earthengine-api** – Google Earth Engine Python API for geospatial analysis
- **geemap** – interactive mapping library that simplifies Earth Engine visualization
- **planet** – Planet SDK for searching and ordering commercial satellite imagery
- **requests** – HTTP library for API calls (e.g., downloading FIRMS data)
- **geopandas** – geospatial extension of pandas for working with vector data
- **shapely** – geometric operations library for creating and manipulating spatial objects

Run the cell below to install or upgrade these packages. The `-q` flag suppresses verbose output, and `--upgrade` ensures you have the latest versions.

**Note:** Some operations in this workflow (particularly Planet API interactions) may benefit from asynchronous execution to handle multiple requests efficiently.

In [1]:
!pip install -q --upgrade earthengine-api leafmap geemap planet requests geopandas shapely

## Imports

## Workflow Overview

This notebook follows a simple pipeline:

1. **Load fire data**: We fetch today's fire detections from NASA's FIRMS dataset via Google Earth Engine
2. **Detect fire clusters**: We group nearby fire pixels together to identify distinct fire events
3. **Find the largest fire**: We identify which fire cluster is the largest
4. **Create a boundary polygon**: We convert the fire cluster into a clean polygon boundary (Area of Interest)
5. **Export as GeoJSON**: We save the boundary as a standard geographic data format

Each step builds on the previous one, creating a streamlined analysis pipeline that takes raw fire detection data and produces an actionable boundary polygon suitable for ordering high-resolution satellite imagery or emergency response planning.

In [2]:
# Core Earth Engine API for accessing and processing geospatial datasets
import ee

# geemap wraps Earth Engine objects for interactive mapping in notebooks
import geemap as geemap

# Planet SDK pieces:
# Auth: used to manage API key sessions
# DataClient: search/catalog queries (e.g., filtering for recent PlanetScope imagery)
# OrdersClient: place orders (e.g., bundle, clip, deliver assets)
from planet import Auth, DataClient, OrdersClient

# requests for simple HTTP downloads (e.g., FIRMS CSV if needed outside EE)
import requests

# geopandas for handling vector data (GeoDataFrame) locally once exported or downloaded
import geopandas as gpd

# shapely provides geometric operations (buffer, area, bounds, etc.)
from shapely.geometry import Polygon, shape, mapping

# Optional: set up Planet authentication (expects your API key in the PL_API_KEY env var)
# auth = Auth.from_env()
# pl_session = auth.create_client_session()
# data_client = DataClient(pl_session)
# orders_client = OrdersClient(pl_session)

# Initialize Earth Engine (authentication handled in a later cell)
# ee.Initialize()


## 1. Authentication with Google Earth Engine

Before you can use Google Earth Engine (GEE), you need to authenticate your account and initialize the API. There are several ways to do this:

*   **Browser Authentication (Recommended for Beginners):** This is the easiest method. GEE will open a browser window to allow you to log in with your Google account.
*   **Service Account:** More suitable for automated scripts and server-side applications. Requires creating a service account in the Google Cloud Console and downloading credentials.

### 1.1 Browser Authentication

The following code snippet initializes the Earth Engine API and authenticates your account using browser authentication. Run this cell first to authenticate.

**What happens when you run this cell:**

1. The code tries to initialize Earth Engine with `ee.Initialize()`
2. If you haven't authenticated yet, it will fail and catch the error
3. It then runs `ee.Authenticate()` which opens your web browser
4. You'll log in with your Google account and grant permissions
5. After successful authentication, it runs `ee.Initialize()` again to connect to Earth Engine
6. Finally, it prints a confirmation message

**Note:** You only need to authenticate once per computer. After that, you can just run `ee.Initialize()` in future sessions.

## Best Practices for Beginners

As you work through this notebook, keep these tips in mind:

- **Run cells in order**: Each cell depends on results from previous cells. Running them out of order may cause errors.
- **Wait for output**: GEE operations can take a few seconds. Wait for the output before running the next cell.
- **Watch the console**: Print statements help you understand what's happening at each step. They're like "signposts" in your analysis.
- **Zoom and pan the maps**: Click and drag to pan, use the mouse wheel or +/- buttons to zoom. This helps you understand what the data actually looks like.
- **Check file paths**: When exporting files, make sure the `output/` directory exists. The code creates it automatically, but it's good to know where it is.


## Understanding Key Concepts

Before we dive into the code, let's understand some core remote sensing and Google Earth Engine concepts that will help you understand what we're doing:

### What is Remote Sensing?
Remote sensing is the science of obtaining information about objects or areas from a distance—typically using satellites. These satellites carry instruments (called "sensors") that measure different wavelengths of light reflected by Earth's surface. By analyzing these different wavelengths, we can identify fires, vegetation, water, and other features.

### What is Google Earth Engine (GEE)?
Google Earth Engine is a free cloud-based platform that holds decades of satellite imagery and makes it easy to analyze this data programmatically. Instead of downloading massive files to your computer, you send your analysis code to Google's servers, which do the heavy lifting and return just the results you need.

### Key GEE Objects You'll See

**ImageCollection**: A collection is like a "folder" of satellite images. The FIRMS dataset is an ImageCollection that contains fire detections from many satellite passes. Each image has multiple "bands" (different measurements of the same area—e.g., brightness temperature in different infrared wavelengths).

**Geometry**: In GEE, geometry defines the area of interest (AOI). It can be a point, line, polygon, or rectangle. We'll use a rectangle covering the entire world.

**Image Operations**: GEE lets you perform mathematical operations on images. For example, `fires.gt(100)` means "create a new image where pixels hotter than 100°C are marked as 1 (true) and colder pixels are marked as 0 (false)."

**Server-side vs. Client-side**: Most GEE operations happen on Google's servers (server-side), which is very fast. When you call `.getInfo()`, you're asking GEE to send the results back to your computer (client-side), which is slower. We try to minimize `.getInfo()` calls.

### What is a Reducer?
A reducer in GEE is a function that combines (reduces) data across space or time. For example, `ee.Reducer.minMax()` finds the minimum and maximum values across all pixels in an area. Think of it as "summarizing" a large image down to one or a few numbers.


In [3]:
ee.Authenticate()
ee.Initialize(project='greenai-hackathon-maples-two')

In [4]:
# ============================================================================
# STEP 1: Load today's active fires from NASA's FIRMS dataset
# ============================================================================

# Load the FIRMS (Fire Information for Resource Management System) ImageCollection
# This dataset contains near-real-time fire detections from satellite sensors
dataset = ee.ImageCollection("FIRMS")

# Create a rectangle covering the entire world
# Coordinates: [west, south, east, north] in degrees (latitude/longitude)
# -180°W to +180°E longitude, -90°S to +90°N latitude
geometry = ee.Geometry.Rectangle([-180, -90, 180, 90])

# Get the most recent image in the collection
# sort(..., False) means sort in descending order (newest first)
# .first() gets just the first (most recent) image
lastimg = dataset.sort('system:time_start', False).first()

# Select the T21 band (brightness temperature in the thermal infrared)
# and create a binary image: pixels hotter than 100°C = 1, cooler pixels = 0
# The gt() function means "greater than"
fires = lastimg.select('T21').gt(100)

# Define how we want to visualize the fire temperature data on the map
# "min" and "max" set the scale—temperatures between 100°C and 500°C will be shown
# "palette" defines the color ramp: cooler fires (100°C) appear yellow,
# medium fires appear orange, and hottest fires appear red
firesVis = {
    "min": 100.0,      # Minimum temperature to display (in °C)
    "max": 500.0,      # Maximum temperature to display (in °C)
    "palette": ["yellow", "orange", "red"]  # Color ramp: cool to hot
}

# Create an interactive geemap Map object centered on the world
m = geemap.Map(center=[0, 0], zoom=2)

# Add the fire temperature layer to the map
# addLayer takes three arguments: the data to display, how to visualize it, and a label
m.addLayer(lastimg.select('T21'), firesVis, 'FIRMS T21 (Fire Temperature)')

# Display the map in the notebook
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

## Clustering Fire Pixels: Finding Groups of Nearby Fires

In remote sensing, fires are often detected as individual pixels (grid cells) on a satellite image. When many pixels cluster together, they likely represent a single large fire event. Our next step is to group connected fire pixels into clusters.

The `connectedPixelCount()` function does this by:
1. Looking at each fire pixel
2. Following "connected" paths to neighboring fire pixels (up, down, left, right, and diagonals)
3. Counting how many pixels form each connected group

This helps us identify that, for example, 47 pixels belong to fire cluster A, 12 pixels belong to fire cluster B, and so on. We'll then visualize these clusters and find the largest one.

In [6]:
# ============================================================================
# STEP 2: Identify fire clusters and find the largest one
# ============================================================================

# connectedPixelCount() counts how many fire pixels are connected to each pixel
# Arguments:
#   - 1000: maximum size to count (any cluster larger than 1000 pixels gets capped at 1000)
#   - True: use 8-directional connectivity (includes diagonals, not just up/down/left/right)
# The result is a new image where each pixel's value = size of its cluster
connectedCount = fires.connectedPixelCount(1000, True)

# Define visualization colors for the cluster sizes
# Small clusters will appear purple, large clusters will appear yellow
conn_vis = {
    'min': 0,          # Smallest possible cluster (0 pixels)
    'max': 500,        # Largest cluster size we expect to display (500 pixels)
    'palette': ['#4B0082', '#6A5ACD', '#8A2BE2', '#DA70D6', '#FFD700']  # Purple→indigo→yellow
}

# Use a reducer to find the minimum and maximum cluster sizes
# A reducer is a function that summarizes many values into one or a few values
# ee.Reducer.minMax() finds both the smallest and largest values in an area
minMax = connectedCount.reduceRegion(
    reducer=ee.Reducer.minMax(),  # Find min and max values
    geometry=geometry,             # Search across the entire world
    scale=1000,                    # Use 1 km resolution pixels for the calculation
    maxPixels=1e9                  # Allow processing up to 1 billion pixels (for performance)
)

# Print the results to see the range of cluster sizes
# Note: .getInfo() brings server-side GEE data to your computer (client-side)
print("Min and Max connected pixel counts:", minMax.getInfo())

# Add the cluster layer to our existing map
# This shows all fire clusters colored by size
m.addLayer(connectedCount, conn_vis, 'Fire Clusters by Size (purple→yellow)')

# Display the updated map
m

Min and Max connected pixel counts: {'T21_max': 130, 'T21_min': 1}


Map(bottom=5056.0, center=[-27.839076094777816, 128.8629269660297], controls=(WidgetControl(options=['position…

In [7]:
# ============================================================================
# STEP 3: Isolate the largest fire cluster
# ============================================================================

# Get the maximum cluster size from our previous calculation
# ee.Number() wraps the value so we can work with it in GEE
T21_max = ee.Number(minMax.get('T21_max'))

# Create a binary image: pixels belonging to the largest cluster = 1, all others = 0
# .eq() means "equals"—so this image marks only pixels equal to T21_max
largest_cluster = connectedCount.eq(T21_max)

# Apply a mask to hide all non-largest-cluster pixels
# .updateMask() makes pixels with value 0 transparent on the map
# Only the largest cluster (value 1) remains visible
largest_mask = largest_cluster.updateMask(largest_cluster)

# Add the largest fire cluster to the map in red for clear visibility
m.addLayer(largest_mask, {'min': 0, 'max': 1, 'palette': ['red']}, 'Largest Fire Cluster')

# Print the size of the largest cluster to the console
print('Size of largest cluster (T21_max):', T21_max.getInfo(), 'connected pixels')
print("Full statistics:", minMax.getInfo())

Size of largest cluster (T21_max): 130 connected pixels
Full statistics: {'T21_max': 130, 'T21_min': 1}


In [8]:
# ============================================================================
# STEP 4: Convert fire cluster pixels into polygon boundaries
# ============================================================================

# Convert the binary fire cluster image to vectors (polygon geometries)
# This turns the pixelated cluster into clean polygon boundaries
# Arguments:
#   - reducer: ee.Reducer.countEvery() counts pixels (used for vector creation)
#   - geometry: search area (entire world)
#   - scale: 1 km resolution pixels
#   - maxPixels: max pixels to process
#   - eightConnected: use 8-directional connectivity (includes diagonals)
#   - geometryType='bb': create bounding boxes (rectangular polygons)
vectors = largest_mask.reduceToVectors(
    reducer=ee.Reducer.countEvery(),
    geometry=geometry,
    scale=1000,
    maxPixels=1e9,
    eightConnected=True,
    geometryType='bb'
)

# For each polygon, calculate its area (in square meters)
# This function will be applied to each feature in the collection
def set_area(feat):
    """Calculate the area of a feature's geometry."""
    geom = feat.geometry()
    # .area(maxError) computes area with specified precision (maxError=1 meter)
    # Server-side operations use non-zero maxError for efficiency
    area_m = geom.area(1)
    # .set() adds a new property to the feature
    return feat.set('area', area_m)

# Apply the set_area function to all polygons
# .map() applies a function to each item in a collection
vectors = vectors.map(set_area)

# Find and extract the polygon with the largest area
# sort(..., False) sorts by area in descending order (largest first)
# .first() gets the first (largest) polygon
largest_feature = ee.Feature(vectors.sort('area', False).first())

# Calculate the area in square kilometers for readability
# .getInfo() brings the server-side calculation result to your computer
area_m = largest_feature.geometry().area(1)
area_m_val = area_m.getInfo()  # Fetch from GEE server to Python
area_km = area_m_val / 1e6     # Convert square meters to square kilometers

print(f'Largest fire area: {area_m_val:,.0f} m² ({area_km:,.1f} km²)')

# Add the polygon boundaries to the map
m.addLayer(vectors, {'color': 'orange'}, 'All cluster polygons')
m.addLayer(ee.FeatureCollection(largest_feature), {'color': 'red', 'fillColor': '00000000'}, 'Largest fire (boundary)')

# Center the map on the largest fire and zoom in
m.centerObject(largest_feature, 10)

# Display the updated map
m

Largest fire area: 171,937,813 m² (171.9 km²)


Map(bottom=5182.0, center=[9.596240817080838, 32.20469464449951], controls=(WidgetControl(options=['position',…

In [9]:
# ============================================================================
# STEP 5: Convert to GeoJSON format (for sharing and interoperability)
# ============================================================================

import json  # Python's built-in module for working with JSON data

# Convert the GEE Feature to a Python dictionary (GeoJSON format)
# GeoJSON is a standard format for geographic data that works across many tools
# .getInfo() brings the server-side feature data to the client (your computer)
largest_feature_info = largest_feature.getInfo()

# Wrap the single feature in a FeatureCollection
# (GeoJSON standard: a collection of features)
feature_collection_geojson = {
    "type": "FeatureCollection",
    "features": [largest_feature_info]
}

# Display the GeoJSON structure in a readable format
# This shows you what the data looks like before saving it
print("GeoJSON Output:")
print(json.dumps(feature_collection_geojson, indent=2))

GeoJSON Output:
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "geodesic": false,
        "type": "Polygon",
        "coordinates": [
          [
            [
              32.14439685572947,
              9.537768932984017
            ],
            [
              32.264992433269406,
              9.537768932984017
            ],
            [
              32.264992433269406,
              9.65471594884454
            ],
            [
              32.14439685572947,
              9.65471594884454
            ],
            [
              32.14439685572947,
              9.537768932984017
            ]
          ]
        ]
      },
      "id": "+23551+8946",
      "properties": {
        "area": 171937812.71457696,
        "count": 130,
        "label": 1
      }
    }
  ]
}


## Export to GeoJSON File

Now we'll save the fire boundary polygon to a GeoJSON file. This file can be:
- Imported into mapping software like QGIS, ArcGIS, or Google Earth
- Used as an Area of Interest (AOI) for ordering satellite imagery
- Shared with other researchers or emergency responders
- Used in web mapping applications

In [ ]:
# ============================================================================
# Save the GeoJSON to a file
# ============================================================================

from pathlib import Path  # Python's cross-platform filesystem path handling

# Create the output directory if it doesn't exist
# parents=True: create parent directories if needed
# exist_ok=True: don't error if directory already exists
output_dir = Path("output")
# Get the date from the image's system:time_start property
# This is the acquisition date of the fire detection data
image_date = ee.Date(lastimg.get('system:time_start'))

# Format the date as YYYY-MM-DD for the filename
# .format() converts the date to a readable string
date_string = image_date.format('YYYY-MM-DD').getInfo()

# Create the output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Specify the GeoJSON file path
# Note: the code updates the filename to include today's date
geojson_path = output_dir / f"{date_string}_largest_fire_cluster.geojson"

# Write the GeoJSON to a file
# json.dump() serializes (converts) the Python dictionary to JSON format
# ensure_ascii=False: preserves special characters (like accents)
# indent=2: makes the file human-readable with 2-space indentation
with geojson_path.open("w", encoding="utf-8") as fp:
    json.dump(feature_collection_geojson, fp, ensure_ascii=False, indent=2)

# Print confirmation message showing where the file was saved
print(f"✓ GeoJSON exported to: {geojson_path.resolve()}")

✓ GeoJSON exported to: /Users/maples/GitHub/google_earth_engine_python_101/notebooks/output/2026-01-28_largest_fire_cluster.geojson
